In [ ]:
from keras.layers import Input, Flatten
from skimage.io import imshow,imread
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers.experimental import AdamW
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import accuracy_score

In [ ]:
base_path='/content/drive/MyDrive/breast cancer'
train_path='/content/drive/MyDrive/train'
val_path='/content/drive/MyDrive/breast cancer'
test_path='/content/drive/MyDrive/test'

In [ ]:
batch_size = 32
target_size=(150, 150)
class_mode='categorical'

In [ ]:
def generate_dataset(path):
    print("\nTotal for trainning : ", end=" ")
    ds = image_dataset_from_directory(path + '/train',
                                      image_size=target_size,
                                          batch_size=batch_size)
    class_names = ds.class_names
    print(class_names)
    num_classes=len(ds.class_names)


    image_data_generator = ImageDataGenerator(rescale = 1./255,
                                      fill_mode = 'nearest',
                                      samplewise_std_normalization=True,
                                      samplewise_center=True,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      )
    test_image_data_generator = ImageDataGenerator(rescale = 1./255)

    print("\nFor Training : ", end=" ")
    train_generator = image_data_generator.flow_from_directory(train_path,
                                                      target_size=target_size,
                                                      batch_size=batch_size,
                                                      class_mode='categorical',
                                                      interpolation="nearest",
                                                      color_mode='rgb')


    print("\nFor Val : ", end=" ")
    validation_generator = image_data_generator.flow_from_directory(val_path,
                                                           target_size=target_size,
                                                           batch_size=batch_size,
                                                           class_mode='categorical',
                                                           shuffle = False,
                                                           interpolation="nearest",
                                                           color_mode='rgb')
    print("\nFor Test : ", end=" ")
    test_generator = test_image_data_generator.flow_from_directory(test_path,
                                                           target_size=target_size,
                                                           batch_size=batch_size,
                                                           class_mode='categorical',
                                                           shuffle=False,
                                                           interpolation="nearest",
                                                           color_mode='rgb')
    plt.figure(figsize=(30, 30))
    for images, labels in ds.take(1):
        for i in range(num_classes):
            ax = plt.subplot(9, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(class_names[labels[i]])
            plt.axis("off")

    return train_generator,validation_generator,test_generator,train_generator.image_shape,class_names,num_classes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
import os


In [ ]:
# Load the pre-trained ResNet50 model without the final classification layer
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers (optional for fine-tuning)
base_model.trainable = False


In [ ]:
# Define data generators for training and validation sets
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

# Define paths to training and validation directories
#print("<<<<<<========= here started =========>>>> ")
#files = os.listdir("/content/drive/MyDrive/train")
#print("No.of files ===> ", len(files))
train_generator = train_datagen.flow_from_directory(
    directory="/content/drive/MyDrive/train",
    target_size=(150, 150),
    batch_size=10,
    class_mode="binary"  # Binary classification (benign/malignant)
)

val_generator = val_datagen.flow_from_directory(
    directory="/content/drive/MyDrive/breast cancer",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)


<<<<<<========= here started =========>>>> 
No.of files ===>  460
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [ ]:
# Add custom layers on top of the pre-trained model for breast cancer classification
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation="sigmoid")(x)  # Sigmoid for probability output

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
# Train the model with the prepared data generators
model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


ValueError: Asked to retrieve element 0, but the Sequence has length 0